In [ ]:
# Instal·lem les lliberies que anem a utilitzar

%pip install transformers datasets evaluate accelerate keras scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import tensorflow as tf
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForCausalLM

In [ ]:
model_id = "ClassCat/gpt2-small-catalan-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = TFAutoModelForCausalLM.from_pretrained(model_id, from_pt=True)

control_code = "alfredo"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/835k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/495k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/923 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/166M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.5.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [ ]:
from datasets import load_dataset
dataset = load_dataset("text", data_dir='/content/drive/MyDrive/audios_peluts/lletres')

def preprocess_function(data):
    data["text"] = f"<|startoftext|><|{control_code}|>{data['text']}<|endoftext|>"

preprocessed_dataset = dataset.map(preprocess_function)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/4138 [00:00<?, ? examples/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4138 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, return_tensors="tf"
)

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
model.fit(x=tf_train_set, epochs=10)

Epoch 1/10
258/258 [==============================] - 57s 119ms/step - loss: 4.8698
Epoch 2/10
258/258 [==============================] - 31s 121ms/step - loss: 4.2820
Epoch 3/10
258/258 [==============================] - 32s 122ms/step - loss: 3.9110
Epoch 4/10
258/258 [==============================] - 31s 121ms/step - loss: 3.5805
Epoch 5/10
258/258 [==============================] - 31s 120ms/step - loss: 3.2820
Epoch 6/10
258/258 [==============================] - 31s 121ms/step - loss: 3.0009
Epoch 7/10
258/258 [==============================] - 31s 120ms/step - loss: 2.7579
Epoch 8/10
258/258 [==============================] - 31s 121ms/step - loss: 2.5405
Epoch 9/10
258/258 [==============================] - 31s 121ms/step - loss: 2.3349
Epoch 10/10
258/258 [==============================] - 31s 122ms/step - loss: 2.1321


In [ ]:
model.save_pretrained("/content/drive/MyDrive/audios_peluts/model")

In [ ]:
from transformers import pipeline
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [ ]:
generator("<|alfredo|>francés")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': '<|alfredo|>francés.aspedronabos.com (2009)>>.aspera.es. ¡eh! algo es...'}]

In [ ]:
generator("<|alfredo|>abuela femer")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': '<|alfredo|>abuela femer.org. ¡eh! ¿el trago? usted sabrà ese hacer als abuelos. todos nuestros con los meus condons. más...'}]

In [ ]:
generator("<|alfredo|>la campana")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': "<|alfredo|>la campana ha tocat le campanar. sona a sol, la nevera acaba. s'ha jubilat. adéu. nou so per sempre. gerard horta.... continua.2018 inmobiliaria coque. con"}]

In [ ]:
generator("<|alfredo|>la vall")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': '<|alfredo|>la vall. 1991.12.20.500.html.eral. --> con xisos. ex., guanajillas. ex.., barón y querido. pero usted, usted'}]

In [ ]:
generator("<|alfredo|>joaquín")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': "<|alfredo|>joaquín.san pedro. ni a ma casa tampoc no n'hi ha ningú en casa. antoni fernández-casero. invito a con vosotros! ¿dónde vas? usted.enos."}]